<a href="https://colab.research.google.com/github/Poojads-udupi/AI--Residency-Gradio/blob/main/weather_assistant24thmay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install openai==0.28 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00


In [3]:
import gradio as gr

In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [5]:
import requests

def get_current_weather(location, unit='celsius'):
    weather_api_key = "a7b109315b6ced6a9cd5177bdb98ca82"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    weather_description = data['weather'][0]['description']
    return {
        "location": location,
        "temperature": data['main']['temp'],
        "weather": weather_description
    }

In [6]:
print(get_current_weather("Abu Dhabi"))

{'location': 'Abu Dhabi', 'temperature': 46.01, 'weather': 'clear sky'}


In [7]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [8]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city, e.g. San Francisco'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}}]

In [9]:
import openai

#Function Definition and Initial Message Handling


def weather_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a weather bot . Answer only in Celsius."})



        # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0,
            messages=messages,
            functions=functions
        )

    #Handling Function Calls and Fetching Weather Data
    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = eval(function_call['arguments'])

        # Fetch weather data using the extracted arguments
        weather_data = get_current_weather(arguments['location'])

        # Append the function call and weather data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "get_current_weather", "content": str(weather_data)})

#magic of llm
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "I'm here to provide weather updates. Please ask me questions related to weather."



In [10]:
weather_chat("i want to go out in the sun in dubai now is it viable")

"The current temperature in Dubai is 44.96°C with a clear sky. It is quite hot and sunny, so you can go out in the sun but make sure to stay hydrated and protect yourself from the sun's rays."

In [11]:
#qc
weather_chat("كم درجة الحرارة في الشارقة؟ ")

'درجة الحرارة في الشارقة هي حالياً تبلغ 45.16 درجة مئوية مع سماء صافية.'

In [14]:
import gradio as gr

# 🔁 Sample city list
top_cities = [
    "New York", "London", "Tokyo", "Paris", "Dubai",
    "Singapore", "Sydney", "Toronto", "Mumbai", "Berlin"
]

# 🌦️ Dummy weather function (replace with real one)


def weather_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a weather bot . Answer only in Celsius."})



        # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0,
            messages=messages,
            functions=functions
        )

# 🖼️ Gradio Blocks Layout
with gr.Blocks(title="Weather Bot") as demo:
    gr.Markdown("## 🌤️ Ask me anything about the weather!")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🌍 City Selection")
            city = gr.Dropdown(choices=top_cities, label="Select a Top City")
            custom_city = gr.Textbox(label="Or Add Your Own City")
            query = gr.Textbox(label="Enter your Weather Question")

        with gr.Column(scale=1):
            gr.Markdown("### ⛅ Weather Result")
            output = gr.Textbox(label="Weather Updates")

    # 🔘 Button and logic
    submit_btn = gr.Button("Get Weather")
    submit_btn.click(
        fn=weather_chat,
        inputs=[city, custom_city, query],
        outputs=output
    )

# 🚀 Launch
demo.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 1 arguments for function <function weather_chat at 0x7d99a3332d40>, received 3.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1036: UserWarning: Expected maximum 1 arguments for function <function weather_chat at 0x7d99a3332d40>, received 3.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6789862d1805e6fc81.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [19]:
import openai
import gradio as gr
import os
from google.colab import userdata

# Load API Key
#load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# 🔁 Sample city list
top_cities = [
    "New York", "London", "Tokyo", "Paris", "Dubai",
    "Singapore", "Sydney", "Toronto", "Mumbai", "Berlin"
]

# 🌦️ Weather Bot Function
def weather_chat(city, custom_city, query):
    selected_city = custom_city if custom_city else city
    user_message = f"What’s the weather in {selected_city}? {query}"

    messages = [
        {"role": "system", "content": "You are a weather bot. Always reply in Celsius."},
        {"role": "user", "content": user_message}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response['choices'][0]['message']['content']

# 🖼️ Gradio Blocks Layout
with gr.Blocks(title="Weather Bot") as demo:
    gr.Markdown("## 🌤️ Ask me anything about the weather!")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🌍 City Selection")
            city = gr.Dropdown(choices=top_cities, label="Select a Top City")
            custom_city = gr.Textbox(label="Or Add Your Own City")
            query = gr.Textbox(label="Enter your Weather Question")

        with gr.Column(scale=1):
            gr.Markdown("### ⛅ Weather Result")
            output = gr.Textbox(label="Weather Updates")

    # 🔘 Button and logic
    submit_btn = gr.Button("Get Weather")
    submit_btn.click(
        fn=weather_chat,
        inputs=[city, custom_city, query],
        outputs=output
    )

# 🚀 Launch
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c6e69631f195853502.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
